In [7]:
import codecs
import glob
import math
import os
from decimal import Decimal
import pandas as pd

In [8]:
data_dir = "時系列/2022xxxxx/*.時系列"
result_dir = "時系列/csv/" + "/".join(data_dir.split("時系列")[1].split("/")[1:-1]) + "/"

round_number = lambda x: round(x, 3)
paths = [p for p in glob.glob(data_dir)]
result = []
# ファイル名を格納するリスト
file_names = []
wavelength_list = []
temp_range = []
df_result = []
num_row_list = []

In [9]:
for path in paths:
    # ファイル名  ex: -60 445.時系列
    file_name = path.split("/")[-1].split(".")[0]
    file_names.append(file_name)
    # ファイル名の整形　ex: -60 445
    file_name2 = file_name.split(".")[0]
    # ファイル名の整形　ex: ['-60', '445']
    file_name3 = file_name2.split(" ")

    with codecs.open(path, "r", "UTF-8", "ignore") as file:
        col_names = ["c{0:02d}".format(i) for i in range(5)]
        df = pd.read_table(file, names=col_names)
        # 測定した波長を抜き出す
        single_g_at = df.iloc[1][0]
        original_wavelength = single_g_at.replace("Trend type: Single g at ", "")
        int_wavelength = float(original_wavelength)
        wavelength = str(math.floor(int_wavelength))

        # TODO ファイル名が特殊な場合はこっち
        new_file_name = file_name
        # file_name = file_name.split(".時系列")[0]
        # new_file_name = new_file_name.split(".時系列")[0]

        # ファイル内容の整形、不要な行の削除、数値型への変換
        df = df.drop("c00", axis=1)
        df = df.rename(
            columns={"c01": "time " + new_file_name, "c02": "abs " + new_file_name}
        )
        # 0~4行目までの削除
        df = df.drop(range(0, 5))
        # 仮のカラムの不要部分削除
        df = df.rename(columns={"c03": "old_name", "c04": "new_name"})
        df["old_name"].iloc[0] = file_name
        df["new_name"].iloc[0] = new_file_name
        # 元データはstr型のためintに変換する必要がある
        alter_int = lambda x: float(x)
        # df['abs '+new_file_name] = df['abs '+new_file_name].map(alter_int)
        df[df.columns[1]] = df[df.columns[1]].map(alter_int)
        # time列の指数表示を解除
        decimal_time = lambda x: Decimal(x)
        df["time " + new_file_name] = df["time " + new_file_name].map(decimal_time)
        # int型に変換
        df["time " + new_file_name] = df["time " + new_file_name].map(alter_int)
        # df.to_csv(result_dir+new_file_name, index=False)

        # ファイル名の変更
        # file_nameが元のファイル名、new_file_nameが新しいファイル名
        os.makedirs(result_dir, exist_ok=True)
        df.to_csv(result_dir + new_file_name + ".csv", index=False)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
